# Utility for reading images

In [1]:
import tensorflow as tf
import os
import pathlib
import random
import PIL

import numpy as np
import IPython.display as display

import cv2


In [2]:
path = '/home/johann/Pictures'
image_dims = [300,300]

In [3]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [500, 500])
    image = image[:,:,::-1]
    #image /= 255.0  # normalize to [0,1] range
    return image

In [4]:
data_root = pathlib.Path(os.path.join(path, 'images'))
image_uris = list(data_root.glob('**/*.jpg'))
image_uris = [str(image) for image in image_uris]

In [5]:
print(image_uris)

['/home/johann/Pictures/images/262985539_1709e54576_z.jpg', '/home/johann/Pictures/images/1045023827_4ec3e8ba5c_z.jpg', '/home/johann/Pictures/images/12283150_12d37e6389_z.jpg', '/home/johann/Pictures/images/25691390_f9944f61b5_z.jpg']


In [6]:
image_ds = tf.data.Dataset.from_tensor_slices(image_uris).map(tf.read_file)

print(image_ds)

with tf.Session().as_default():
    tfrec = tf.data.experimental.TFRecordWriter(os.path.join(path, 'dest_images', 'images.tfrec'))
    
    tfrec.write(image_ds).run()
    
    image_ds = tf.data.TFRecordDataset(os.path.join(path, 'dest_images', 'images.tfrec')).map(preprocess_image)
    it = image_ds.make_one_shot_iterator()
    image_tensor = it.get_next()
    image = tf.Session().run(image_tensor)
    
    print(image.shape)
    i=0
    while True:
        image = tf.Session().run(image_tensor)
        i+=1
        cv2.imwrite(os.path.join(path, 'dest_images', '{}.jpg'.format(str(i))), image)
    


<DatasetV1Adapter shapes: (), types: tf.string>
(500, 500, 3)


KeyboardInterrupt: 